# Fairness evaluation of `bert-base-uncased`
This notebook will call all fairness metrics in the Biased Rulers package. First we start with some preliminary imports...

In [4]:
import os
os.chdir("../")
import numpy as np
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
import torch
import sys
sys.path.append("workspace/thesis/biased-rulers")
os.chdir("../")
!ls
from biased_rulers.metrics import seat, lpbs, crowspairs
#os.chdir("../")

NGC-DL-CONTAINER-LICENSE    etc		 lib32	 opt   srv	 var
bin			    get-pip.py	 lib64	 proc  start.sh  workspace
boot			    home	 libx32  root  sys
cuda-keyring_1.0-1_all.deb  jupyter.log  media	 run   tmp
dev			    lib		 mnt	 sbin  usr


ModuleNotFoundError: No module named 'biased_rulers'

## Define and download model

In [2]:

model_type = "bert-base-multilingual-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_type)
model = AutoModel.from_pretrained(model_type)
print(f"Loaded {model_type}")


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


ImportError: cannot import name 'DistilRobertaTokenizer' from 'transformers' (/usr/local/lib/python3.10/dist-packages/transformers/__init__.py)

## WEAT-based tests
In this section, we run our experiments for the WEAT-based metrics. Our Biased Rulers package supports SEAT (May et al., 2019) and two variants by Lauscher et al. (2021) and Tan et al. (2019).

In [7]:

attribute_template = "This is the _."
target_template = "This is the _."

results = seat.seat_test(attribute_template, target_template, tokenizer, model)
score = np.fromiter(results.values(), dtype=float).mean()
print(score)

0.4365659724426786


In [6]:
results = seat.lauscher_et_al_test(attribute_template, target_template, tokenizer, model)
score = np.fromiter(results.values(), dtype=float).mean()
print(score)

0.4501284455471183


In [45]:
results = seat.tan_et_al_test(attribute_template, target_template, tokenizer, model)
score = np.fromiter(results.values(), dtype=float).mean()
print(score)

0.8573153343423693


## Log probability bias score
In this section, we run the LPBS experiments. This requires a slightly different model setup, so we just initialize it again.

In [8]:
model = AutoModelForMaskedLM.from_pretrained(model_type) # SEAT and other methods expect a different model

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
results = lpbs.lpbs_test("","", tokenizer, model)
print(results)

TypeError: lpbs_test() takes 2 positional arguments but 4 were given

## CrowS-Pairs test
Finally, we test an extrinsic measure.

In [5]:
crows_score = crowspairs.evaluate(tokenizer, model)

/workspace/thesis/biased-rulers/biased_rulers/data/crowspairs.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_data = df_data.append(df_item, ignore_index=True)
/workspace/thesis/biased-rulers/biased_rulers/data/crowspairs.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_data = df_data.append(df_item, ignore_index=True)
/workspace/thesis/biased-rulers/biased_rulers/data/crowspairs.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_data = df_data.append(df_item, ignore_index=True)
/workspace/thesis/biased-rulers/biased_rulers/data/crowspairs.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_data = df_data.append(

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.